# Get all open questions

In [1]:
import load_secrets
load_secrets.load_secrets()

In [2]:
from forecasting_tools import MetaculusApi, ApiFilter, DataOrganizer
from datetime import datetime
import asyncio, time, joblib

api_filter = ApiFilter(
    allowed_statuses=["open"],
)

all_open = []

In [3]:
for chunk in range(0,1000):
    print('chunk starting', chunk)
    offset = chunk * MetaculusApi.MAX_QUESTIONS_FROM_QUESTION_API_PER_REQUEST
    questions, success = MetaculusApi._grab_filtered_questions_with_offset(api_filter, offset)
    if success:
        all_open.extend(questions)
        print(len(all_open))
        time.sleep(1.1)
    else:
        print('done')
        break

chunk starting 0
100
chunk starting 1
200
chunk starting 2
300
chunk starting 3
400
chunk starting 4
500
chunk starting 5
600
chunk starting 6
700
chunk starting 7
800
chunk starting 8
900
chunk starting 9
1000
chunk starting 10
1100
chunk starting 11
1200
chunk starting 12
1300
chunk starting 13
1400
chunk starting 14
1500
chunk starting 15
1600
chunk starting 16
1700
chunk starting 17
1800
chunk starting 18
1900
chunk starting 19
2000
chunk starting 20
2100
chunk starting 21
2200
chunk starting 22
2300
chunk starting 23
2385
chunk starting 24
done


In [4]:
joblib.dump(all_open, 'all_open.joblib')

['all_open.joblib']

In [5]:
questions = joblib.load('all_open.joblib')

In [6]:
from flatten_dict import flatten_dict

In [7]:
import pandas as pd
df = pd.DataFrame([flatten_dict(vars(q)) for q in questions])

In [10]:
df1 = df[df['num_forecasters'] > 40]

In [11]:
len(df1)

1049

In [13]:
!mkdir open

In [15]:
question_ids = df1.id_of_question.values

In [16]:
from tqdm import tqdm

In [17]:
for question in tqdm(questions):
    if question.id_of_question in question_ids:
        fn=f'open/{question.id_of_question}.joblib'
        joblib.dump(question, fn)


100%|██████████████████████████████████████| 2385/2385 [00:06<00:00, 347.70it/s]


In [19]:
question = joblib.load('open/37391.joblib')

In [20]:
question

BinaryQuestion(question_text='Will Florida have 14.0 million or more residents living in drought on June 28, 2025?', id_of_post=38085, page_url='https://www.metaculus.com/questions/38085', id_of_question=37391, state=<QuestionState.OPEN: 'open'>, num_forecasters=81, num_predictions=83, resolution_criteria='This question resolves as **Yes** if Drought Monitor at [this link](https://www.drought.gov/states/florida) lists greater than or equal to 14.0 million Florida residents in areas of drought, when the link is accessed by Metaculus on or after June 28, 2025. If the number is below that, this question resolves as **No**.', fine_print="The question will resolve as soon as the page is accessed by Metaculus Admins on the resolution date. However, if there are issues accessing the page, then resolution will wait up until the end of the tournament's scoring at which point if Metaculus has still not been able to access the resolution source, this question will be **annulled**.", background_in

In [ ]:
import glob, joblib

num_questions = (36000, 40000)

start, end = num_questions
fns = glob.glob('open/*.joblib')

fn = fns[0]

fns = [(int(fn.split('/')[1].split('.')[0]), fn) for fn in fns]

fns[0]

fns = [fn for id,fn in fns if start <= id <= end]

len(fns)

questions = [joblib.load(fn) for fn in fns]

len(questions)

set([type(x) for x in questions])